<a href="https://colab.research.google.com/github/patrickbryant1/MoLPC/blob/master/MoLPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [153]:
#Install dependencies
%pip install biopython
%pip install JAX
%pip install dm-haiku
%pip install ml-collections

UsageError: Line magic function `%conda` not found.


In [141]:
import shutil
shutil.rmtree('/content/MoLPC', ignore_errors=True)

In [142]:
!git clone https://github.com/patrickbryant1/MoLPC.git

Cloning into 'MoLPC'...
remote: Enumerating objects: 189, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 189 (delta 59), reused 166 (delta 39), pack-reused 0
Receiving objects: 100% (189/189), 34.72 MiB | 30.21 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [111]:
#Run the pipeline
import sys
from google.colab import files
sys.path.insert(0,'/content/MoLPC/src')
#@title Default title text
ID = "1A8R" #@param {type:"string"}
SUBSIZE = 3 #@param {type:"integer"} #What order the subcomplexes should be (2 or 3)
#Get the csvs
import pandas as pd
DATADIR='/content/MoLPC/data/test/'
USEQS=pd.read_csv(DATADIR+ID+'_useqs.csv')
CHAINS=pd.read_csv(DATADIR+ID+'_chains.csv')
INTERACTIONS='' #Leave empty if the interactions are not known - here they are not used. See the file $DATADIR/$ID'_ints.csv' to how to supply such a file
GET_ALL=1 #If to get all interactions (1) or not (0) - when the interactions are known
MSADIR=DATADIR+'/'
OUTDIR="/content/gdrive/MyDrive/"

In [101]:
#Mount the drive to be able to save files
from google.colab import drive
drive.mount('/content/gdrive') #All the output will be written here

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [112]:
#########Step1: MSA PIPELINE#########
#Now, a default MSA is read in - no search is performed here
#Write the Paired and Block Diagonalized MSAs to predict sub-components
from preprocess import preprocess_colab
preprocess_colab.create_folder_structure(MSADIR, ID, OUTDIR, USEQS, INTERACTIONS, CHAINS, GET_ALL, SUBSIZE)

Creating all interactions of size 3 ...


In [113]:
#########Step2: FOLDING PIPELINE#########
#Create structure dir
STRUCTURE_DIR=OUTDIR+"AF/"
os.mkdir(STRUCTURE_DIR)


In [137]:
#Get the sub_ids and lengths
import glob
files = glob.glob(OUTDIR+'*.fasta')
sub_ids = {}
for filename in files:
  with open(filename, 'r') as file:
    for line in file:
      line = line.rstrip()[1:].split('|')
      sub_ids[line[0]]=line[-1].split('-')[:-1]
      break
print(sub_ids)

{'1A8R_1-1-1': ['221', '221']}


In [135]:
#Get the AF2 params
import shutil
PARAMS=STRUCTURE_DIR+'params/'
if not os.path.exists(PARAMS):
  os.mkdir(PARAMS)
  !wget https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar 
  shutil.move('/content/alphafold_params_2021-07-14.tar', PARAMS)

In [140]:
#Predict the subcomponents
##### AF2 CONFIGURATION #####
PARAM=STRUCTURE_DIR
PRESET='full_dbs' #Choose preset model configuration - no ensembling (full_dbs) and (reduced_dbs) or 8 model ensemblings (casp14).
MAX_RECYCLES=10 #max_recycles (default=3)
MODEL_NAME='model_1' #model_1_ptm

#Go through all subcomponents and predict their structure
for sub_id in sub_ids:
  ####Get fasta file####
  FASTAFILE=OUTDIR+sub_id+'.fasta'
  ####Get chain break#### Note! This is now set for trimer subcomponents
  CB=','.join(sub_ids[sub_id])
  ####Get MSAs####
  #HHblits paired
  PAIREDMSA=OUTDIR+sub_id+'_paired.a3m'
  ##HHblits block diagonalized
  BLOCKEDMSA=OUTDIR+sub_id+'_blocked.a3m'
  MSAS=",".join([PAIREDMSA,BLOCKEDMSA]) #Comma separated list of msa paths
  print(sub_id,CB, MSAS)

1A8R_1-1-1 221,221 /content/gdrive/MyDrive/1A8R_1-1-1_paired.a3m,/content/gdrive/MyDrive/1A8R_1-1-1_blocked.a3m


In [150]:
sys.path.insert(0,'/content/MoLPC/src/AF2')
from AF2 import run_alphafold_colab

ModuleNotFoundError: ignored

In [ ]:
$BASE/src/AF2/run_alphafold.py \
                --fasta_paths=$FASTAFILE \
                --msas=$MSAS \
                --chain_break_list=$CB \
                --output_dir=$STRUCTURE_DIR \
                --model_names=$MODEL_NAME \
                --data_dir=$PARAM \
                --fold_only \
                --uniref90_database_path=$HHBLITSDB \
                --mgnify_database_path=$HHBLITSDB \
                --bfd_database_path=$HHBLITSDB \
                --uniclust30_database_path=$HHBLITSDB \
                --pdb70_database_path=$HHBLITSDB \
                --template_mmcif_dir=$HHBLITSDB \
                --obsolete_pdbs_path=$HHBLITSDB \
                --preset=$PRESET \
                --max_recycles=$MAX_RECYCLES

done